## Global Dam Watch Attribution

In [2]:
arcpy.env.workspace = r"C:\Users\olivoshj\Desktop\ArcGIS_Projects\Dam_Project_Aux\Dam_Project_Aux.gdb"

In [6]:
input_ranges = "Extant_Native_Ranges_Clipped"
# Global Dam Watch Consensus database downloaded from https://www.globaldamwatch.org/database
gdw_fc = "GDW_barriers_v1_0"

In [7]:
# change -99 for nulls
with arcpy.da.UpdateCursor(gdw_fc, ['CATCH_SKM','DIS_AVG_LS','AREA_SKM']) as cursor:
    for row in cursor:
        for i in [0,1,2]:
            if row[i] == -99:
                row[i] = None
                cursor.updateRow(row)

In [8]:

# select small to medium dams and obstructions
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=gdw_fc,
    selection_type="NEW_SELECTION",
    where_clause="ORIG_SRC LIKE '%GOODD%' Or ORIG_SRC LIKE '%GROD%' Or ORIG_SRC LIKE '%JRC%'",
    invert_where_clause=None
)

# sum catchment areas, discharge and reservoir area values overlapping each range
arcpy.analysis.SummarizeWithin(
    in_polygons=input_ranges,
    in_sum_features=gdw_fc,
    out_feature_class=f"{input_ranges}_GDW_Small",
    keep_all_polygons="KEEP_ALL",
    sum_fields="CATCH_SKM Sum;DIS_AVG_LS Sum;AREA_SKM Sum",
    sum_shape="ADD_SHAPE_SUM"
)

# adjust field names (temporary)
arcpy.management.AlterFields(
    in_table=f"{input_ranges}_GDW_Small",
    field_description=f"sum_CATCH_SKM smallDams_CATCH_SKM smallDams_CATCH_SKM DOUBLE # true #;sum_DIS_AVG_LS smallDams_DIS_AVG_LS smallDams_DIS_AVG_LS DOUBLE # true #;sum_AREA_SKM smallDams_AREA_SKM smallDams_AREA_SKM DOUBLE # true #;Point_Count smallDams_count smallDams_count LONG # true #"
)

# join back to first feature class
arcpy.management.JoinField(
    in_data=input_ranges,
    in_field="sci_name",
    join_table=f"{input_ranges}_GDW_Small",
    join_field="sci_name",
    fields=f"smallDams_CATCH_SKM;smallDams_DIS_AVG_LS;smallDams_AREA_SKM;smallDams_count"
)

arcpy.management.Delete(
    in_data=fr"{arcpy.env.workspace}\{input_ranges}_GDW_Small"
)

<Result 'true'>

In [9]:

# select large dams
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=gdw_fc,
    selection_type="NEW_SELECTION",
    where_clause="ORIG_SRC LIKE '%GRanD%'",
    invert_where_clause=None
)

# sum catchment areas, discharge and reservoir area values overlapping each range
arcpy.analysis.SummarizeWithin(
    in_polygons=input_ranges,
    in_sum_features=gdw_fc,
    out_feature_class=f"{input_ranges}_GDW_Large",
    keep_all_polygons="KEEP_ALL",
    sum_fields="CATCH_SKM Sum;DIS_AVG_LS Sum;AREA_SKM Sum",
    sum_shape="ADD_SHAPE_SUM"
)

# adjust field names (temporary)
arcpy.management.AlterFields(
    in_table=f"{input_ranges}_GDW_Large",
    field_description=f"sum_CATCH_SKM largeDams_CATCH_SKM largeDams_CATCH_SKM DOUBLE # true #;sum_DIS_AVG_LS largeDams_DIS_AVG_LS largeDams_DIS_AVG_LS DOUBLE # true #;sum_AREA_SKM largeDams_AREA_SKM largeDams_AREA_SKM DOUBLE # true #;Point_Count largeDams_count largeDams_count LONG # true #"
)

# join back to first feature class
arcpy.management.JoinField(
    in_data=input_ranges,
    in_field="sci_name",
    join_table=f"{input_ranges}_GDW_Large",
    join_field="sci_name",
    fields=f"largeDams_CATCH_SKM;largeDams_DIS_AVG_LS;largeDams_AREA_SKM;largeDams_count"
)

arcpy.management.Delete(
    in_data=fr"{arcpy.env.workspace}\{input_ranges}_GDW_Large"
)

<Result 'true'>

In [11]:
# export table to csv
arcpy.conversion.ExportTable(
    in_table=input_ranges,
    out_table=r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\GDW_Attributes.csv",
    field_mapping=f'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99;smallDams_CATCH_SKM "smallDams_CATCH_SKM" true true false 8 Double 0 0,First,#,{input_ranges},smallDams_CATCH_SKM,-1,-1;smallDams_DIS_AVG_LS "smallDams_DIS_AVG_LS" true true false 8 Double 0 0,First,#,{input_ranges},smallDams_DIS_AVG_LS,-1,-1;smallDams_AREA_SKM "smallDams_AREA_SKM" true true false 8 Double 0 0,First,#,{input_ranges},smallDams_AREA_SKM,-1,-1;smallDams_count "smallDams_count" true true false 4 Long 0 0,First,#,{input_ranges},smallDams_count,-1,-1;largeDams_CATCH_SKM "largeDams_CATCH_SKM" true true false 8 Double 0 0,First,#,{input_ranges},largeDams_CATCH_SKM,-1,-1;largeDams_DIS_AVG_LS "largeDams_DIS_AVG_LS" true true false 8 Double 0 0,First,#,{input_ranges},largeDams_DIS_AVG_LS,-1,-1;largeDams_AREA_SKM "largeDams_AREA_SKM" true true false 8 Double 0 0,First,#,{input_ranges},largeDams_AREA_SKM,-1,-1;largeDams_count "largeDams_count" true true false 4 Long 0 0,First,#,{input_ranges},largeDams_count,-1,-1'
)

<Result 'R:\\FWL\\Arismendi-Lab\\Andres\\Gilbert_Freshwater_Fish_Analysis\\Revised_Analysis_NatureCommunications\\Input_datasets\\GDW_Attributes.csv'>